### 1. Импорты и настройка окружения

In [1]:
# Installations

# ! pip install transformers sentencepiece catboost torch scikit-learn pandas scikit-learn pyarrow fastparquet ipywidgets widgetsnbextension
 

In [2]:
import zipfile
import pandas as pd
import json

import torch
from transformers import AutoTokenizer, AutoModel

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn import preprocessing

from catboost import Pool, cv

from statistics import mean


In [3]:
# Extention for CatBoost visualisation

# ! jupyter nbextension enable --py widgetsnbextension

### 2. Исследовательский анализ данных

Распаковка архива

In [4]:
# archive_path = 'internship_2023.zip'
# directory_to_extract_to = 'KE_internship_2023'
# 
# with zipfile.ZipFile(archive_path, 'r') as zip_ref:
#     zip_ref.extractall(directory_to_extract_to)

**Описание датасета**

In [5]:
df = pd.read_parquet('KE_internship_2023/train.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91120 entries, 0 to 99992
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     91120 non-null  int64  
 1   category_id    91120 non-null  int64  
 2   sale           91120 non-null  bool   
 3   shop_id        91120 non-null  int64  
 4   shop_title     91120 non-null  object 
 5   rating         91120 non-null  float64
 6   text_fields    91120 non-null  object 
 7   category_name  91120 non-null  object 
dtypes: bool(1), float64(1), int64(3), object(3)
memory usage: 5.6+ MB


Было решено использовать только табличные данные. На это есть ряд причин:

1) Теоретически, возможны случаи того, что товар со схожим описанием может иметь довольно разный внешний вид, мы от этого не застрахованы. Тогда CNN может неправильно классифицировать такие вещи. Отсутствие таких случаев в тренировочном датасете негативно скажется на точности предсказаний. Следовательно -хорошая текстовая модель будет более робастной. 
2) Также Object Classification модель будет плохо определять редкие классы в нашем случае, ведь наш тренировочный датасет сильно несбалансирован (показано дальше).
3) В телеграм-чат стажировки скинули репозиторий о классификации продуктов маркетплейса при помощи мультимодального подхода (с использованием как текста, так и картинок) - https://github.com/1sh1vam/E-commerce-products-classification-using-images-and-text?ysclid=lf9m2l93ft95384990. После этой публикации было решено отказаться от фьюза текста и изображений.
4) **Личная причина** - На своём текущем месте работы я решал Computer Vision задачи (такие как Object Detection), поэтому решение NLP-задачи мне кажется более интересным.

Давайте посмотрим на изначальные фичи и поправим индексацию, для удобства (отсутсвует индекс 2).

In [6]:
# Важные фичи - shop_id, что-то из text_fields

df.head()

,product_id,category_id,sale,shop_id,shop_title,rating,text_fields,category_name
0,325286,12171,False,9031,Aksik,5.0,"{""title"": ""Зарядный кабель Borofone BX1 Lightn...",Все категории->Электроника->Смартфоны и телефо...
1,888134,14233,False,18305,Sela,5.0,"{""title"": ""Трусы Sela"", ""description"": ""Трусы-...",Все категории->Одежда->Женская одежда->Белье и...
3,1267173,13429,False,16357,ЮНЛАНДИЯ канцтовары,5.0,"{""title"": ""Гуашь \""ЮНЫЙ ВОЛШЕБНИК\"", 12 цветов...",Все категории->Хобби и творчество->Рисование->...
4,1416943,2789,False,34666,вася-nicotine,4.0,"{""title"": ""Колба для кальяна Крафт (разные цве...",Все категории->Хобби и творчество->Товары для ...
5,1058275,12834,False,26389,Lim Market,4.6,"{""title"": ""Пижама женская, однотонная с шортам...",Все категории->Одежда->Женская одежда->Домашня...


In [7]:
df = df.reindex(range(len(df)),method='ffill')
df.head()

,product_id,category_id,sale,shop_id,shop_title,rating,text_fields,category_name
0,325286,12171,False,9031,Aksik,5.0,"{""title"": ""Зарядный кабель Borofone BX1 Lightn...",Все категории->Электроника->Смартфоны и телефо...
1,888134,14233,False,18305,Sela,5.0,"{""title"": ""Трусы Sela"", ""description"": ""Трусы-...",Все категории->Одежда->Женская одежда->Белье и...
2,888134,14233,False,18305,Sela,5.0,"{""title"": ""Трусы Sela"", ""description"": ""Трусы-...",Все категории->Одежда->Женская одежда->Белье и...
3,1267173,13429,False,16357,ЮНЛАНДИЯ канцтовары,5.0,"{""title"": ""Гуашь \""ЮНЫЙ ВОЛШЕБНИК\"", 12 цветов...",Все категории->Хобби и творчество->Рисование->...
4,1416943,2789,False,34666,вася-nicotine,4.0,"{""title"": ""Колба для кальяна Крафт (разные цве...",Все категории->Хобби и творчество->Товары для ...


В нашем датасете присутсвует 874 класса.

In [8]:
df['category_id'].nunique()

874

In [9]:
occur = df['category_id'].value_counts()
occur

11937    6646
14922    3741
13143    1455
13651    1450
12604    1231
         ... 
14033       1
12836       1
11549       1
11875       1
12808       1
Name: category_id, Length: 874, dtype: int64

Я пошёл по лёгкому пути - выбросил строчки из классов, которые встречаются < 10 раз. Ведь должно быть так, чтобы каждый класс попал в тренировочный и тестовый датасет или в каждый фолд кросс-валидации. Более корректной была бы генерация новый данных для редких классов.

In [10]:
rare_labels = occur[occur < 10].index
indexes_to_drop = df[df['category_id'].isin(rare_labels)].index
indexes_to_drop

Int64Index([  134,   229,   405,   455,   491,   524,   529,   776,   817,
             1203,
            ...
            90219, 90686, 90692, 90719, 90856, 90964, 91002, 91027, 91046,
            91071],
           dtype='int64', length=883)

In [11]:
df.drop(indexes_to_drop, inplace=True)
# reindexing
df = df.reindex(range(len(df)),method='ffill')
df['category_id'].nunique()

728

У нас осталось 728 уникальных классов

### 3. Обработка данных

Наиболее значимые фичи - shoз_title и поле **title** в text_fields. Было решено в дальнейшем использовать только их.

In [12]:
# Retrieving info from text_fields

def parse_text(df: pd.DataFrame):
  product_title = []
  shop_title = []
  
  for index, row in df.iterrows():
    # processing json
    json_dict = json.loads(row['text_fields'].lower())
    title_value = json_dict['title']
    product_title.append(title_value)
    # processing shop_title
    shop_title.append(row['shop_title'].lower())

  product_title = pd.Series(product_title)
  shop_title = pd.Series(shop_title)
  return product_title, shop_title

product_title, shop_title = parse_text(df)
product_title.head()

0    зарядный кабель borofone bx1 lightning для айф...
1                                           трусы sela
2                                           трусы sela
3    гуашь "юный волшебник", 12 цветов по 35 мл, бо...
4               колба для кальяна крафт (разные цвета)
dtype: object

In [13]:
shop_title.head()

0                  aksik
1                   sela
2                   sela
3    юнландия канцтовары
4          вася-nicotine
dtype: object

Кодировка категорий.
* LabelEncoding был использован для упорядочивания ID категорий (от 0 до 727). 
* categ_dict - словарь, мапящий новый category_id к category_name. Нужен для визуальной проверки работы модели.
* new_to_old_dict - словарь, мапящий новый category_id к старому category_id.

In [14]:
old_categs = df['category_id'].unique()
category_name = df['category_name'].unique()

df['category_id'] = preprocessing.LabelEncoder().fit_transform(df['category_id'])
new_categs = df['category_id'].unique()

categ_dict = {k:v for k,v in zip(new_categs, category_name)}
new_to_old_dict = {k:v for k,v in zip(new_categs, old_categs)}

In [15]:
df = df.drop(['product_id', 'sale', 'shop_id', 'rating', 'text_fields', 'category_name'], axis = 1)

df['shop_title'] = shop_title
df['product_title'] = product_title

df.tail(5)

,category_id,shop_title,product_title
90232,360,el'kar,мантоварка/пароварка алюминиевая 6 л с 3 сетка...
90233,689,iservice.market,стекло ceramics матовое для tecno camon 15 air
90234,102,pigtail,"кронштейн для тв uniteki fm1625 32""-55"""
90235,168,o2body,грабер щипцы для захвата и чистки раковины
90236,46,vazan,"набор для творчества ""эпоксидная смола"""


Как мы видим, пустых строчек нет.

In [16]:
df.isna().sum().sum()

0

### 4. Реализация простого бейзлайна

Для начала было решено сделать простейшее решение. Выбор пал на метод, описанный в этой статье - https://towardsdatascience.com/classifying-marketplace-inventory-at-scale-with-machine-learning-99e69eac585e

В бейзлайне:

1) Через CountVectorize формируется словарь (Bag of Words). Он просто:
    * Токенизирует строки.
    * Считает количество каждого токена в корпусе.
2) Через этот словарь создаёт sparce-вектора строк. Например:
    * Слово "кальян" будет иметь вектор (0, 322), что значит - вектор, где все элементы равны 0, кроме 322 (1). Этот вектор sparce, так как он почти полностью состоит из 0.
3) Классификация через Naive Bayes.

In [17]:
df_copy = df 
df_copy = df_copy.drop(['shop_title'], axis = 1)

vocabulary = CountVectorizer()
# К сожалению, не получилось обучить на 2 столбцах
vocabulary.fit(df_copy['product_title'])
print(f'Размер словаря: {len(vocabulary.vocabulary_)}')

Размер словаря: 38847


In [18]:
counts = vocabulary.transform(df_copy['product_title'])
counts

<90237x38847 sparse matrix of type '<class 'numpy.int64'>'
	with 584274 stored elements in Compressed Sparse Row format>

In [19]:
n_train = int(.8 * df_copy.shape[0])


x_train = counts[:n_train]
y_train = df_copy['category_id'][:n_train]

x_test = counts[n_train:]
y_test = df_copy['category_id'][n_train:]

x_train

<72189x38847 sparse matrix of type '<class 'numpy.int64'>'
	with 467768 stored elements in Compressed Sparse Row format>

In [20]:
nb = MultinomialNB()

nb.fit(x_train, y_train)
predictions = nb.predict(x_test)

f1_Bayes = f1_score(y_test, predictions, average='weighted')
print(f1_Bayes)


0.645030911730545


Weighter f1 получился = 0.665

**Описать, почему данный метод не очень**

### 5. Генерация эмбеддингов

Для векторизации текста было решено использовать dense векторное представление текста: 
1) чтобы не генерировать вектора очень большой размерности (как, например, через CountVectorizer).
2) SOTA методы генерируют именно dense эмбеддинги.

Был использован претренированный rubert-tiny2 от Сбера, так как:
1) Он знает как русский, так английский язык.
2) Эмбеддинг сайз = 312, а эти минимальный размер эмбеддинга среди Бертов. Это уменьшит вычислительную нагрузку.
3) Энкодер РуБерта - это трансформер, а за счёт self-attention он понимает контекст и берёт во внимание порядок слов. 
4) Также он может векторизовать как 1 слово, так и предложение. Для колонки 'shop_title' это не важно, но для 'product_title' - это очень важно.

In [21]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
model.cuda()

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
   

In [22]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [23]:
embedding_size = embed_bert_cls('привет мир', model, tokenizer).shape[0]
embedding_size

312

In [24]:
def text2colums(features):
  # Создание пустых Датафреймов
  product_title = pd.DataFrame(index=features.index, columns = range(embedding_size))
  shop_title = pd.DataFrame(index=features.index, columns = range(embedding_size, 2*embedding_size))

  for i in range(len(features)):
    product_title.loc[features.index[i], :] = embed_bert_cls(features['product_title'][i], model, tokenizer)
    shop_title.loc[features.index[i], :] = embed_bert_cls(features['shop_title'][i], model, tokenizer)
  return product_title, shop_title


In [25]:
product_title, shop_title = text2colums(df)

product_title = pd.DataFrame(product_title, index=df.index, columns = range(embedding_size))
product_title.head()

,0,1,2,3,4,5,6,7,8,9,...,302,303,304,305,306,307,308,309,310,311
0,-0.041848,0.076797,0.100111,-0.09684,-0.0175,0.031053,-0.038998,-0.040375,-0.03768,-0.041215,...,0.03641,0.004447,0.028502,-0.038608,-0.023932,-0.012764,0.007445,0.045618,0.045941,-0.062663
1,0.077617,-0.029442,0.000839,-0.048024,0.010647,-0.006344,0.080962,0.033694,-0.016182,-0.03961,...,0.061222,0.007703,0.014565,0.036552,0.032607,-0.001435,-0.00623,0.016611,0.037598,-0.102271
2,0.077617,-0.029442,0.000839,-0.048024,0.010647,-0.006344,0.080962,0.033694,-0.016182,-0.03961,...,0.061222,0.007703,0.014565,0.036552,0.032607,-0.001435,-0.00623,0.016611,0.037598,-0.102271
3,0.105562,0.031073,-0.032013,-0.114572,-0.030688,0.040483,-0.060621,0.031279,-0.055251,-0.003542,...,0.069968,-0.028857,0.020296,-0.017557,0.027912,0.017539,-0.010598,0.016,0.048476,-0.021955
4,0.067184,-0.078257,-0.002021,-0.032678,0.0144,0.027183,0.003262,0.048267,-0.109209,-0.060674,...,0.051381,0.032925,0.010762,-0.042912,0.01832,0.020478,-0.047669,0.038493,0.097577,-0.125223


In [26]:
shop_title.head()

,312,313,314,315,316,317,318,319,320,321,...,614,615,616,617,618,619,620,621,622,623
0,0.026618,-0.046738,0.01004,-0.038673,0.039722,0.017055,0.086188,0.022375,-0.002136,0.016785,...,0.014339,0.006808,0.019619,0.000222,0.069489,-0.015718,-0.038993,-0.03254,0.070624,-0.086863
1,0.047774,-0.007012,0.010186,-0.057968,-0.007213,-0.015189,0.058021,0.017656,-0.029094,-0.021018,...,0.049047,0.008838,-0.022514,0.027404,0.02931,-0.003394,0.00442,-0.039563,0.020289,-0.062201
2,0.047774,-0.007012,0.010186,-0.057968,-0.007213,-0.015189,0.058021,0.017656,-0.029094,-0.021018,...,0.049047,0.008838,-0.022514,0.027404,0.02931,-0.003394,0.00442,-0.039563,0.020289,-0.062201
3,0.081229,-0.025745,0.043016,-0.036166,0.015317,-0.003419,0.06258,0.010517,-0.05752,-0.052432,...,0.030071,0.044538,-0.014533,0.022788,0.060086,-0.014246,-0.027965,0.07792,0.065011,-0.131839
4,0.020314,0.039335,0.009987,-0.093424,-0.006235,0.039141,0.036472,-0.007236,0.014387,-0.029035,...,0.027334,0.042659,-0.048443,0.019821,0.063454,-0.010788,-0.039814,0.023349,0.110247,-0.080109


### 6. Обучение основной модели

Очень хотелось решить данную задачу без всемогущего чёрного ящика - нейронных сетей. Рассматривались следующие алгоритмы классического машинного обучения с учётом того, что наша задача - мультиклассовая классификация:

1) KNN - так как планировалось использовать 2 rubert эмбеддинга для тренировки (размерность = 624), то без уменьшения размерности (например, PCA), данная модель бы пострадала бы от Curse of Dimensionality. Но условный PCA не был применён из-за страха того, что он сильно исказит "смысл" эмбеддингов.

2) K-means для классификации - не использовался из-за Curse of Dimensionality, аналогично с KNN.

3) Naive Bayes - **интуиция** подсказала, что лучше использовать другой классификатор.

4) Logistic Regression - хоть это и бинарный классификатор, но переформулировав задачу в One vs one или One vs rest, им можно решить и мультикласс. Но **интуиция** подсказала то, что эта модель не сможет решить такую сложную задачу.

5) Остались деревья решений и SVM.

**Выбор остановлен на градиентном бустинге CatBoost**, так как:

1) Сама библиотека CatBoost очень удобная и позваляет **StraightForward** запустить кросс-валидация и вывести ошибку.
2) Градиентный бустинг традиционно хорошо показывает себя на мультиклассовой классификации табличных данных

SVM (есть быть точнее, Support Vector Classifier) тоже можно было использовать ведь это мощная модель. Но в отличии от CatBoost, его нужно было бы дольше тюнить (пробовать One vs One, One vs Rest, возможно помимо линейнего кернела нужно было бы попробовать и другие)

In [27]:
df_copy = df 
df_copy

,category_id,shop_title,product_title
0,205,aksik,зарядный кабель borofone bx1 lightning для айф...
1,620,sela,трусы sela
2,620,sela,трусы sela
3,458,юнландия канцтовары,"гуашь ""юный волшебник"", 12 цветов по 35 мл, бо..."
4,60,вася-nicotine,колба для кальяна крафт (разные цвета)
...,...,...,...
90232,360,el'kar,мантоварка/пароварка алюминиевая 6 л с 3 сетка...
90233,689,iservice.market,стекло ceramics матовое для tecno camon 15 air
90234,102,pigtail,"кронштейн для тв uniteki fm1625 32""-55"""
90235,168,o2body,грабер щипцы для захвата и чистки раковины


In [28]:
df_copy = df_copy.drop(['product_title', 'shop_title'], axis = 1)
df_copy = pd.concat([df_copy, product_title, shop_title], axis=1)

In [29]:
# Random shuffle 
df_copy = df_copy.sample(frac=1).reset_index(drop=True)
df_copy

,category_id,0,1,2,3,4,5,6,7,8,...,614,615,616,617,618,619,620,621,622,623
0,451,0.041565,-0.006827,0.021578,-0.059066,0.00019,0.05974,-0.000039,0.057721,-0.004248,...,0.087466,0.077071,0.005514,-0.018843,0.035283,0.013954,-0.05117,0.02564,0.058478,-0.099549
1,604,-0.086439,0.008445,0.02383,-0.126691,-0.011536,0.043355,-0.008233,0.027485,-0.080029,...,0.066933,-0.000366,-0.039678,-0.006019,0.028291,0.036701,-0.000778,0.059175,0.011477,0.009664
2,163,-0.022545,0.03679,0.034935,-0.060531,-0.005727,0.051085,-0.081528,0.0188,-0.049508,...,0.030189,0.030399,-0.017827,-0.020975,0.040112,-0.011632,-0.036375,0.045179,0.057489,-0.056625
3,74,0.048612,-0.043489,0.017148,-0.126079,0.045456,0.012962,-0.028805,0.009134,-0.061795,...,0.055504,0.006097,-0.046257,0.003595,0.054635,-0.019156,-0.010001,-0.006771,0.063839,-0.070384
4,165,0.046508,-0.035187,0.006396,-0.049864,0.022464,0.010209,0.029745,0.038138,-0.028309,...,-0.016821,0.023795,0.01829,-0.023281,0.020119,-0.012231,-0.064141,0.053578,0.023031,-0.119314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90232,512,0.05115,0.011102,0.0342,-0.001359,0.041342,-0.01237,-0.027774,0.033896,-0.041717,...,0.008113,0.036417,-0.03205,0.016818,0.007465,-0.016967,-0.037875,-0.040717,0.128553,-0.076932
90233,309,0.058573,-0.034381,0.059354,-0.043286,0.010083,0.048982,-0.029273,0.043703,-0.074774,...,0.038842,0.016217,-0.07791,-0.026662,0.071448,-0.032572,0.005547,-0.004953,0.042386,-0.124937
90234,51,0.048028,-0.068317,0.035106,-0.017742,0.029954,-0.009983,0.013147,0.077841,-0.10914,...,0.039915,0.069705,-0.039512,-0.020819,0.026866,0.018403,-0.026464,0.029391,0.031261,-0.107029
90235,51,0.016765,-0.034757,0.07935,-0.068233,0.039645,-0.021102,-0.006219,0.019417,-0.063767,...,0.030172,0.060051,-0.025614,-0.012518,0.058907,0.023986,-0.000153,-0.008302,0.056754,-0.075775


5-fold Кросс Валидация градиентного бустинга

In [31]:
cv_dataset = Pool(data=df_copy.drop(['category_id'], axis=1),
                  label=df_copy['category_id'])

params = {"iterations": 500,
          "learning_rate": 0.3,
          "random_seed": 1,
          "loss_function": 'MultiClass',
          "task_type": "GPU",
          "use_best_model": True}


scores = cv(cv_dataset,
            params,
            iterations = 500,
            fold_count=5, 
            shuffle = False,
            plot=True,
            return_models = True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]
0:	learn: 7.1332965	test: 7.1035587	best: 7.1035587 (0)	total: 2.44s	remaining: 20m 18s
1:	learn: 7.6320177	test: 7.6758172	best: 7.1035587 (0)	total: 4.92s	remaining: 20m 24s
2:	learn: 7.7554954	test: 7.7935621	best: 7.1035587 (0)	total: 7.3s	remaining: 20m 8s
3:	learn: 8.2382416	test: 8.2877906	best: 7.1035587 (0)	total: 9.8s	remaining: 20m 15s
4:	learn: 7.9798164	test: 8.0767377	best: 7.1035587 (0)	total: 12.2s	remaining: 20m 8s
5:	learn: 7.9256581	test: 7.9745744	best: 7.1035587 (0)	total: 14.7s	remaining: 20m 12s
6:	learn: 7.9753225	test: 8.0741964	best: 7.1035587 (0)	total: 17.2s	remaining: 20m 8s
7:	learn: 7.6208904	test: 7.7409999	best: 7.1035587 (0)	total: 19.6s	remaining: 20m 2s
8:	learn: 7.3374122	test: 7.4838734	best: 7.1035587 (0)	total: 22s	remaining: 19m 59s
9:	learn: 7.0266149	test: 7.2064523	best: 7.1035587 (0)	total: 24.5s	remaining: 19m 59s
10:	learn: 6.8297577	test: 7.0349210	best: 7.0349210 (10)	total: 26.9s	remaining: 19m 58s
11:	learn: 6.59

Было обучено 5 моделей градиентного бустинга. Давайте посчитаем weighted f1 score для каждой модели. Используем каждый из 5 фолдов для оценки метрики. Далее посмотрим на среднее и минимальное значение метрики для каждой из модели.

In [32]:
n_test = int(.2 * df_copy.shape[0])

models = scores[1]
models_f1 = {key: [] for key in range(5)}

for i in range(5):
    x_test = df_copy.drop(['category_id'], axis = 1)[i*n_test : (i+1) * n_test]
    y_test= df_copy['category_id'][i * n_test : (i+1) * n_test]

    for i in range(5):
        predictions = [j.argmax() for j in models[i].predict(x_test)]
        models_f1[i].append(f1_score(y_test, predictions, average='weighted'))

models_f1_mean = {k: mean(v) for k,v in models_f1.items()}
models_f1_min = {k: min(v) for k,v in models_f1.items()}

In [34]:
models_f1_min

{0: 0.7385642168714862,
 1: 0.7939797319860074,
 2: 0.7931506359338611,
 3: 0.7901515808233524,
 4: 0.7787363651773512}

Лучший weighted_f1_score ≈ 0.79. Для финального предикта было решено взять модель с **лучшим** значением метрик и функции ошибки - модель с индексом [1]. 

### Финальный инференс

In [34]:
test_df = pd.read_parquet('KE_internship_2023/test.parquet')
test_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16860 entries, 1 to 24995
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   product_id   16860 non-null  int64  
 1   sale         16860 non-null  bool   
 2   shop_id      16860 non-null  int64  
 3   shop_title   16860 non-null  object 
 4   rating       16860 non-null  float64
 5   text_fields  16860 non-null  object 
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 806.8+ KB


In [35]:
product_title, shop_title = parse_text(test_df)

# Формирование датафрейма

test_df = test_df.drop(['sale', 'shop_id', 'rating', 'text_fields'], axis = 1)

test_df['shop_title'] = shop_title
test_df['product_title'] = product_title
test_df = test_df.reindex(range(len(test_df)),method='ffill')
# Ой, строчка ниже сломала тип данных в product_id
test_df = pd.DataFrame.dropna(test_df)

test_df.head()

,product_id,shop_title,product_title
1,1997646.0,di-di market,стекло пленка керамик матовое honor 50 lite 10...
2,927375.0,visionstore,"проводные наушники с микрофоном jack 3.5, ios,..."
3,1921513.0,fornails,"декоративная табличка ""правила кухни"", подстав..."
4,1668662.0,моя кухня,"подставка под ложку керамическая, подложка ""кл..."
5,1467778.0,style icon,"футболка женская с принтом, премиальный хлопок"


In [36]:
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
model.cuda()

# Шафл 
test_df = test_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.reindex(range(len(test_df)),method='ffill')

# Генерация эмбеддингов
product_title, shop_title = text2colums(test_df)

# Фикс типов данных 
test_df = test_df.astype({'product_id': 'int64'})

test_df_copy = test_df.drop(['product_title', 'shop_title'], axis = 1)
test_df_copy = pd.concat([test_df_copy, product_title, shop_title], axis=1)


Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [66]:
test_df.head()

,product_id,shop_title,product_title
0,876120,magichobby,пряжа gazzal jeans 58% хлопок 42% акрил 50г 17...
1,1461170,happy room,коробка складная картонная
2,1767349,smile of milady,тапочки комнатные женские
3,798579,outside the box,"печь и ложка для плавки сургуча, 3 цвета / сур..."
4,433998,лайма,"насадка моп для швабры самоотжимной ""бабочка"" ..."


In [53]:
final_model = models[1]
final_predictions = [j.argmax() for j in final_model.predict(test_df_copy)]


In [65]:
final_names = pd.Series([categ_dict[i] for i in final_predictions], name = "predicted_name")
present = pd.concat([test_df['product_title'],final_names], axis = 1)
present.head()

,product_title,predicted_name
0,пряжа gazzal jeans 58% хлопок 42% акрил 50г 17...,Все категории->Хобби и творчество->Рукоделие->...
1,коробка складная картонная,Все категории->Товары для дома->Товары для пра...
2,тапочки комнатные женские,Все категории->Обувь->Женская обувь->Домашняя ...
3,"печь и ложка для плавки сургуча, 3 цвета / сур...",Все категории->Хобби и творчество->Рукоделие->...
4,"насадка моп для швабры самоотжимной ""бабочка"" ...",Все категории->Товары для дома->Хозяйственные ...


In [67]:
print(present['product_title'][2], present["predicted_name"][2])

тапочки комнатные женские Все категории->Обувь->Женская обувь->Домашняя обувь


In [68]:
print(present['product_title'][4], present["predicted_name"][4])

насадка моп для швабры самоотжимной "бабочка" 603607, спонж/микрофибра 26 см, на липучке, любаша, 603608 Все категории->Товары для дома->Хозяйственные товары->Инвентарь для уборки->Швабры и аксессуары->Швабры


Как видно из ячеек выше - на первый взгляд наша модель выдаёт адекватные предикты.

In [63]:
final_id = pd.Series([new_to_old_dict[i] for i in final_predictions], name = "predicted_category_id ")
for_submission = pd.concat([test_df['product_id'],final_id], axis = 1)
for_submission.to_parquet(path = 'result.parquet')

In [69]:
for_submission.head()

,product_id,predicted_category_id
0,876120,2754
1,1461170,11757
2,1767349,13263
3,798579,2775
4,433998,14874


### Спасибо большое KE за интересное задание!